In [1]:
import numpy as np
import pandas as pd
import ipynb.fs.full.get_artists_info_api as getter

In [38]:
tracks_id = []
for i in range(len(getter.tracks)):
    tracks_id.append(getter.tracks[i]['id'])

tracks_id = list(dict.fromkeys(tracks_id))

In [100]:
track_features_list = []

for i in range(len(tracks_id)):
    track_features_list.append(getter.spotify.audio_features(tracks_id[i]))

In [101]:
#there are some tracks without their audio features so we need to remove them from the list
track_features_list = list(filter(lambda x: x != [None], track_features_list))

In [102]:
for i in range(len(track_features_list)):
    track_features_list[i] = [dict(list(filter(lambda x: type(x[1]) != str, list(track_features_list[i][0].items()))))]